In [1]:
import sys, os, logging, torch
from torch import optim
sys.path.append('../src')
import zipfile
## import data_loader
from data_loader import *

## import model
from model import U2NET
from loss_fun import muti_dice_loss_fusion, muti_bce_loss_fusion

## loggin configuration
logging.basicConfig(filename='log.log',level=logging.DEBUG)

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [2]:
model_name = 'u2net_512'
model_dir = os.path.join(os.getcwd(), 'saved_models', model_name + os.sep)

fold=1;train=True # train=False (means validation)

ds = HuBMAPDataset(fold=fold, train=True, tfms=get_aug())
dl = DataLoader(ds,batch_size=bs,shuffle=False,num_workers=NUM_WORKERS)
# ------- 3. define model --------
net = U2NET(3, 1) # 3layers in input 1 layer in output
if torch.cuda.is_available():
    net.cuda()

# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

# Load from checkpoint
# if 0:
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     checkpoint = torch.load('saved_models/u2net/u2net_bce_itr_2000_train_0.529323_tar_0.071989_fold_2.pth',
#                            map_location=torch.device(device))
#     net.load_state_dict(checkpoint)

fold:  1 train:  True
---define optimizer...


In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F
# ------- 5. training process --------
print("---start training...")
epoch_num = 20 # num of epoc to train
ite_num = 0 # iteration number
running_loss = 0.0 
running_tar_loss = 0.0 
ite_num4val = 0 # iter_num for validation
save_frq = 2000 # save the model every 2000 iterations
train_num=0
for epoch in range(0, epoch_num):
    net.train()
    for i, data in enumerate(dl):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data[0], data[1]
        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
        
        ######  L O S S Starts Here ###### ###### 
        ###### ###### ###### ###### ###### ###### 
        ## binary cross entropy loss
        loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v,logging)
        ## binary cross entropy + dice loss
        #loss2, loss = muti_dice_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v,logging)
        ###### ###### ###### ###### ###### ###### 
        ###### ###### ###### ###### ###### ###### 
        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.item()
        running_tar_loss += loss2.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        #print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        #epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
        logging.debug("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
            epoch + 1, epoch_num, (i + 1) * bs, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
        
        if ite_num % save_frq == 0:
            os.makedirs(name='saved_models/u2net_512',exist_ok=True)
            torch.save(net.state_dict(), model_dir + model_name+"_bce_itr_%d_train_%3f_tar_%3f_fold_%1d.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val,fold))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0


In [ ]:
1

In [ ]:
break here

In [3]:
class Model_pred:
    def __init__(self, model, dl, tta:bool=True, half:bool=False):
        self.model = model
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        self.model.eval()
        name_list = self.dl.dataset.fnames
        count=0
        with torch.no_grad():
            for x,y in iter(self.dl):
                #x = x.cuda()
                if self.half: x = x.half()
                p = self.model(x)
                py = torch.sigmoid(p[0]).detach()
                if self.tta:
                    #x,y,xy flips as TTA
                    flips = [[-1],[-2],[-2,-1]]
                    for f in flips:
                        p = self.model(torch.flip(x,f))
                        p = torch.flip(p[0],f)
                        py += torch.sigmoid(p).detach()
                    py /= (1+len(flips))
                if y is not None and len(y.shape)==4 and py.shape != y.shape:
                    py = F.upsample(py, size=(y.shape[-2],y.shape[-1]), mode="bilinear")
                py = py.permute(0,2,3,1).float().cpu()
                batch_size = len(py)
                for i in range(batch_size):
                    taget = y[i].detach().cpu() if y is not None else None
                    yield py[i],taget,name_list[count]
                    count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)

        
def save_img(data,name,out):
    data = data.float().cpu().numpy()
    img = cv2.imencode('.png',(data*255).astype(np.uint8))[1]
    out.writestr(name, img)

In [4]:
MODELS=[
    'saved_models/u2net_512/u2net_512_bce_itr_12000_train_0.265627_tar_0.033775_fold_0.pth',
    'saved_models/u2net_512/u2net_512_bce_itr_10000_train_0.287260_tar_0.035382_fold_0.pth',
    'saved_models/u2net_512/u2net_512_bce_itr_8000_train_0.343669_tar_0.038781_fold_0.pth',
    'saved_models/u2net_512/u2net_512_bce_itr_6000_train_0.405051_tar_0.043267_fold_0.pth'
    ]
# MODELS=[
#     'saved_models/u2net/u2net_bce_itr_6000_train_0.334218_tar_0.042558_fold_0.pth',
#     'saved_models/u2net/u2net_bce_itr_6000_train_0.331338_tar_0.040776_fold_1.pth',
#     'saved_models/u2net/u2net_bce_itr_6000_train_0.243286_tar_0.032711_fold_2.pth',
#     'saved_models/u2net/u2net_bce_itr_6000_train_0.257452_tar_0.034796_fold_3.pth'
#     ]
fold11=['1200','1000','8000','6000']

In [6]:
for xxx in range(4):
    fold1=fold11[xxx]
    device ='cpu'# torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    checkpoint = torch.load(MODELS[xxx],map_location=torch.device(device))
    net.load_state_dict(checkpoint)
    ds_v = HuBMAPDataset(fold=0, train=False)
    dl = DataLoader(ds_v,batch_size=bs,shuffle=False,num_workers=NUM_WORKERS)
    mp = Model_pred(net,dl)
    kkk=mp.__iter__()
    zip_name='val_masks_tta_bce_512_0_'+str(fold1)+'.zip'
    with zipfile.ZipFile(zip_name, 'a') as out:
        for py,taget,nam in kkk:
            img = cv2.imencode('.png',(np.array(py)*255).astype(np.uint8))[1]
            out.writestr(nam, img)


fold:  0 train:  False


KeyboardInterrupt: 

In [ ]:
1

In [ ]:
# nam='1e2425f28_469.png'
# nam='1e2425f28_431.png'
# nam='2f6ecfcdf_382.png'
# nam='aaa6a05cc_100.png'
nam='54f2eec69_140.png'
nam='54f2eec69_186.png'
print(nam)
img=cv2.imread('../input/hubmap-256x256/masks/'+nam)
img=img*255
Image.fromarray(img)

In [ ]:
nam

In [ ]:
# 'val_masks_tta_0/aaa6a05cc_100.png'
'val_masks_tta_dicebce/54f2eec69_140.png'

In [ ]:
fold=0
# imgp=cv2.imread('val_masks_tta_dicebce'+str(fold)+'/'+nam)
imgp=cv2.imread('val_masks_tta_'+str(fold)+'/'+nam)
# imgp=cv2.imread('val_masks_tta_dicebce/'+nam)
imgp=imgp*255
imgp[imgp>135]=255
Image.fromarray(imgp)

In [ ]:
img = cv2.imencode('.png',(np.array(py)*255).astype(np.uint8))[1]

In [ ]:
with zipfile.ZipFile('val_masks_tta.zip', 'a') as out:
    out.writestr(nam, img)

In [ ]:
# plt.figure(figsize=(16,16))
# for i,(img,mask) in enumerate(zip(imgs,masks)):
#     img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
#     plt.subplot(8,8,i+1)
#     plt.imshow(img,vmin=0,vmax=255)
#     plt.imshow(mask.squeeze().numpy(), alpha=0.03)
#     plt.axis('off')
#     plt.subplots_adjust(wspace=None, hspace=None)

In [ ]:
# plt.figure(figsize=(32,32))
# for i,(img,mask) in enumerate(zip(imgs,masks)):
#     img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
#     plt.subplot(16,16,2*i+1)
#     plt.imshow(img,vmin=0,vmax=255)
#     plt.axis('off')
#     plt.subplots_adjust(wspace=None, hspace=None)
    
#     plt.subplot(16,16,2*i+2)
#     plt.imshow(mask.squeeze().numpy(), alpha=1)
#     plt.axis('off')
#     plt.subplots_adjust(wspace=None, hspace=None)

In [ ]:
# plt.figure(figsize=(32,32))
# for i,(img,mask) in enumerate(zip(imgs,masks)):
#     img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
#     plt.subplot(16,16,2*i+1)
#     plt.imshow(img,vmin=0,vmax=255)
#     plt.axis('off')
#     plt.subplots_adjust(wspace=None, hspace=None)
    
#     plt.subplot(16,16,2*i+2)
#     plt.imshow(mask.squeeze().numpy(), alpha=1)
#     plt.axis('off')
#     plt.subplots_adjust(wspace=None, hspace=None)

In [ ]:
# pd.read_csv('../input/hubmap-kidney-segmentation/train.csv')
# id	encoding
# 0	2f6ecfcdf	296084587 4 296115835 6 296115859 14 296147109...
# 1	aaa6a05cc	30989109 59 31007591 64 31026074 68 31044556 7...
# 2	cb2d976f4	78144363 5 78179297 15 78214231 25 78249165 35...
# 3	0486052bb	101676003 6 101701785 8 101727568 9 101753351 ...
# 4	e79de561c	7464094 14 7480273 41 7496453 67 7512632 82 75...
# 5	095bf7a1f	113430380 22 113468538 67 113506697 111 113544...
# 6	54f2eec69	124601765 36 124632133 109 124662536 147 12469...
# 7	1e2425f28	49453112 7 49479881 22 49506657 31 49533433 40...

In [ ]:
1